### 加载文件

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../datasets/d2l-zh-pytorch.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

797


In [6]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

动手学深度学习
Release2.0.0
AstonZhang,ZacharyC.Lipton,MuLi,andAlexanderJ.Smola
Aug18,2023

{'producer': 'xdvipdfmx (20210609)', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-18T21:21:47+00:00', 'title': '动手学深度学习', 'author': 'Aston Zhang, Zachary C. Lipton, Mu Li, and Alexander J. Smola', 'source': '../datasets/d2l-zh-pytorch.pdf', 'total_pages': 797, 'page': 0, 'page_label': '1'}


### 内容分块

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

1221

### 文本向量化

In [12]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="bge-m3",  # 确保 Ollama 已下载该模型（运行 `ollama pull bge-m3`）
    base_url="http://localhost:11434",  # 注意去掉 /v1
)

C:\Users\tiansz\AppData\Local\Temp\ipykernel_11948\2095470469.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


In [13]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"生成的长度为 {len(vector_1)}\n")
print(vector_1[:10])

生成的长度为 1024

[-0.29000037908554077, -0.32046055793762207, -0.7968214750289917, -0.860930323600769, -0.7268328070640564, -0.05147669464349747, 0.6050247550010681, -1.122937798500061, 0.01522921770811081, 0.5210712552070618]


In [14]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [15]:
ids = vector_store.add_documents(documents=all_splits)

### 文本召回

In [ ]:
results = vector_store.similarity_search(
    "什么是深度学习"
)

print(results[0])

page_content='小结
• 机器学习研究计算机系统如何利用经验（通常是数据）来提高特定任务的性能。它结合了统计学、数据
挖掘和优化的思想。通常，它是被用作实现人工智能解决方案的一种手段。
• 表示学习作为机器学习的一类，其研究的重点是如何自动找到合适的数据表示方式。深度学习是通过
学习多层次的转换来进行的多层次的表示学习。
• 深度学习不仅取代了传统机器学习的浅层模型，而且取代了劳动密集型的特征工程。
• 最近在深度学习方面取得的许多进展，大都是由廉价传感器和互联网规模应用所产生的大量数据，以
及（通过GPU）算力的突破来触发的。
• 整个系统优化是获得高性能的关键环节。有效的深度学习框架的开源使得这一点的设计和实现变得非
常容易。
练习
1. 你当前正在编写的代码的哪些部分可以“学习” ， 即通过学习和自动确定代码中所做的设计选择来改进？
你的代码是否包含启发式设计选择？
2. 你遇到的哪些问题有许多解决它们的样本，但没有具体的自动化方法？这些可能是使用深度学习的主
要候选者。
3. 如果把人工智能的发展看作一场新的工业革命，那么算法和数据之间的关系是什么？它类似于蒸汽机
和煤吗？根本区别是什么？
4. 你还可以在哪里应用端到端的训练方法，比如图1.1.2、物理、工程和计量经济学？
Discussions35
35 https://discuss.d2l.ai/t/1744
38 1. 引言' metadata={'producer': 'xdvipdfmx (20210609)', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-18T21:21:47+00:00', 'title': '动手学深度学习', 'author': 'Aston Zhang, Zachary C. Lipton, Mu Li, and Alexander J. Smola', 'source': '../datasets/d2l-zh-pytorch.pdf', 'total_pages': 797, 'page': 55, 'page_label': '38', 'start_index': 0}


In [17]:
results = vector_store.similarity_search_with_score("BERT模型")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.65121952598117

page_content='#@save
class BERTModel(nn.Module):
"""BERT模型"""
def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
ffn_num_hiddens, num_heads, num_layers, dropout,
max_len=1000, key_size=768, query_size=768, value_size=768,
hid_in_features=768, mlm_in_features=768,
nsp_in_features=768):
super(BERTModel, self).__init__()
self.encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape,
ffn_num_input, ffn_num_hiddens, num_heads, num_layers,
dropout, max_len=max_len, key_size=key_size,
query_size=query_size, value_size=value_size)
self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
nn.Tanh())
self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
self.nsp = NextSentencePred(nsp_in_features)
def forward(self, tokens, segments, valid_lens=None,
pred_positions=None):
encoded_X = self.encoder(tokens, segments, valid_lens)
if pred_positions is not None:
mlm_Y_hat = self.mlm(encoded_X, pred_positions)
else:
mlm_Y_hat = None
# 用于下一句预

In [18]:
embedding = embeddings.embed_query("BERT模型")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='#@save
class BERTModel(nn.Module):
"""BERT模型"""
def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
ffn_num_hiddens, num_heads, num_layers, dropout,
max_len=1000, key_size=768, query_size=768, value_size=768,
hid_in_features=768, mlm_in_features=768,
nsp_in_features=768):
super(BERTModel, self).__init__()
self.encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape,
ffn_num_input, ffn_num_hiddens, num_heads, num_layers,
dropout, max_len=max_len, key_size=key_size,
query_size=query_size, value_size=value_size)
self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
nn.Tanh())
self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
self.nsp = NextSentencePred(nsp_in_features)
def forward(self, tokens, segments, valid_lens=None,
pred_positions=None):
encoded_X = self.encoder(tokens, segments, valid_lens)
if pred_positions is not None:
mlm_Y_hat = self.mlm(encoded_X, pred_positions)
else:
mlm_Y_hat = None
# 用于下一句预测的多层感知机分类器的隐藏层，0是“<cls>”标